In [1]:
import torch
torch.cuda.is_available()

True

In [19]:
a = torch.randn(5,3)
a

tensor([[ 1.4175, -1.6927,  0.3563],
        [-0.2580, -1.4198,  2.0559],
        [-0.8909,  0.8677, -0.0892],
        [-0.1695, -0.8295,  2.3951],
        [ 0.3504,  0.0501,  1.3034]])

In [20]:
u,s,v = torch.svd(a)

In [21]:

s = torch.diag(s)

In [22]:
u.matmul(s).matmul(v.T)

tensor([[ 1.4175, -1.6927,  0.3563],
        [-0.2580, -1.4198,  2.0559],
        [-0.8909,  0.8677, -0.0892],
        [-0.1695, -0.8295,  2.3951],
        [ 0.3504,  0.0501,  1.3034]])

In [24]:
u = u[:,:2]
s = torch.diag(s[:2])
v = v.T[:2,:]
u.matmul(s).matmul(v)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 3x3)

In [30]:
v = v.T[:2,:]
print(u,s,v,sep="\n")

tensor([[ 0.3533, -0.7808],
        [ 0.6234,  0.1540],
        [-0.1665,  0.4703],
        [ 0.6166,  0.3408],
        [ 0.2802,  0.1713]])
tensor([3.9250, 2.2465])
tensor([[ 0.1228, -0.5414,  0.8317],
        [-0.6958,  0.5506,  0.4612]])


In [52]:
from model.nwp_lstm import LSTM
from utils import Config
config = Config(json_path="config/reddit_config.json")
model = LSTM(
            config.vocab_size,
            config.embedding_dim,
            config.hidden_dim,
            config.num_layers,
            config.dropout_rate,
            config.tie_weights,
        )